# Data cleaning and reformating

In [ ]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset\


import pandas as pd
import numpy as np
import os

from random import shuffle, seed

In [ ]:
header = ['TimeSec', 'Sensor', 'Quatx', 'Quaty', 'Quatz', 'Quat0', 'Heading',
       'Pitch', 'Roll', 'LinAccx', 'LinAccy', 'LinAccz', 'Vbat', 'Accx',
       'Accy', 'Accz', 'Gyrox', 'Gyroy', 'Gyroz']

In [ ]:
### Take a look at one dataset
# file_path = "./spinetrack data/Task_Drilling_Stoop.csv"
# df = pd.read_csv(file_path, error_bad_lines=False)
# df.columns = header

# df["Activity"] = [1 for i in range(len(df))]
# df

In [ ]:
# index_lst = []
# index_dict = {}
# # for i in sorted(os.listdir('./Spinetrack Data/data/Alex_data/Processed_redone')):
# for i in sorted(os.listdir('./Spinetrack Data/data/Alex_data/Processed')):
#     if i.endswith(".csv") and i != 'super_features.csv': 
#         # should we combinme same activities together?
#         act_lst = i.split('.')[0].split('_')
#         if len(act_lst) > 1 and act_lst[1] == 'OneH':
#             act = act_lst[0] + '_' + act_lst[1]
#         else:
#             act = act_lst[0]
#         #act = i.split('.')[0]
#         index_lst.append(act)

# # delete multiple items
# index_set = set(index_lst)
# index_set.remove('Static')
# index_set.remove('L')
# index_set.remove('L_OneH')
# index_set.add('Static_Stoop')
# index_set.add('Lifting')
# index_set.add('Lifting_OneH')

# index_dict = {act:i for i, act in enumerate(index_set)}

In [ ]:
# print(index_dict)

In [ ]:
index_dict = {'Pulling_OneH': 0, 'Overhead': 1, 'Pulling': 2, 'Sitting': 3, 
              'Lifting': 4, 'Crawling': 5, 'Standing': 6, 'Carrying': 7, 
              'Walking': 8, 'Pushing': 9, 'Reaching': 10, 'Static_Stoop': 11, 
              'Kneeling': 12, 'Lifting_OneH': 13, 'Crouching': 14}

In [ ]:
directory = "./Spinetrack Data/data/"
file = []

for d in sorted(os.listdir(directory)):
    if d != '.DS_Store' and d != 'Niraj_data':
        files = directory + d + "/Processed_redone"
#         files = directory + d
        name = []
        try:
            for f in sorted(os.listdir(files)):
                if f.endswith(".csv") and f != 'super_features.csv' and not f.startswith('._'): 
                    name.append(files + "/" + f)
            file.append(name)
            print("%s number of activity: %s" %(d, len(name)))
        except:
            pass
#file

In [ ]:
frame = []
for person in file:
    for file_path in person:
        activity_name = ''
        activity_lst = file_path.split("/")[-1].split('.')[0].split('_')
        
        if activity_lst[0] == 'L' and activity_lst[1] == 'OneH':
            activity_name = 'Lifting_OneH'
        elif activity_lst[0] == 'L' and activity_lst[1] != 'OneH':
            activity_name = 'Lifting'
        elif activity_lst[0] == 'Static':
            activity_name = 'Static_Stoop'
        elif activity_lst[0] == 'Pulling' and activity_lst[1] == 'OneH':
            activity_name = 'Pulling_OneH'
        else:
            activity_name = activity_lst[0]
#         print(activity_name)
        print("processing: ", file_path)
        if activity_name in index_dict.keys():
            df = pd.read_csv(file_path, error_bad_lines=False, skiprows=5)
            df.columns = header
            df["activity"] = [index_dict[activity_name] for i in range(len(df))]
            frame.append(df)
            
            # mkdir if not exist. Save to local csv file
#             if not (os.path.exists('./Spinetrack Data/Yibin_Processed/' + category + folder_name)):
#                 os.makedirs('./Spinetrack Data/Yibin_Processed/' + category + folder_name)
#             csv_name = '/' + activity_name + '.csv'
#             folder_name = file_path.split("/")[-3] # person's name 
#             category = 'data/' # data or task
#             file_name = './Spinetrack Data/Yibin_Processed/' + category + folder_name + csv_name
#             df.to_csv(file_name) # save csv processed file to local
            
            #print(file_path)
        

In [ ]:
# the correct dataset should have 986091 rows and 20 columns
data_df = pd.concat(frame).dropna()
data_df

In [ ]:
# result_df = alex_data_df
# result_df
result_df = data_df

In [ ]:
# # the sensor label index range from 0 to 7 
# sensor_data = []
# for i in range(0, 8):
#     df = alex_data_df.where(alex_data_df['Sensor'] == i).dropna()
#     sensor_data.append(df)
# result_df = pd.concat(sensor_data).reset_index(drop=True)
# result_df

---
# Data processing and deep learning model

### Hyperparameter
window_size: The number of timesteps in one window (e.g. how many rows in one window).

channel: The number of features in one window. Similar to image channels (RGB).

batch_size: The numebr of windows in one batch.

learning_rate: How fast the model learns.

In [ ]:
window_size = 45
channel = 1
batch_size = 32
learning_rate = 0.001

In [ ]:
# Process original dataset, create windows (window_size samples(rows), about 1 second)
data = []
window = 1
while window*window_size < len(result_df):
    data_window = result_df[(window - 1)*window_size:window*window_size]
    data.append(data_window.values)
    window += 1
#data

In [ ]:
len(data)

In [ ]:
# delete window if multiple activities and sensors presents or if the time is not increasing
cleaned_data = []
for i in data:
    previous_activity = -1
    previous_sensor = -1
    previous_time = -1
    for j in i:
        current_activity = j[19]
        current_sensor = j[1]
        current_time = j[0]
        if (previous_activity != -1) and (current_activity != previous_activity):
            print("data contains different activities! Window droped")
            break
#         elif (previous_sensor != -1) and (current_sensor != previous_sensor):
#             print("data contains different sensors! Window droped")
#             break
        elif (previous_time != -1) and (current_time < previous_time):
            print("data contains different inconsistent time! Window droped")
            break
        else:
            previous_activity = current_activity
            previous_sensor = current_sensor
            previous_time = current_time
    else:
        cleaned_data.append(i)

In [ ]:
len(cleaned_data)

In [ ]:
cleaned_data[0].shape

In [ ]:
# shuffle the data
seed(101)
shuffle(cleaned_data)
#cleaned_data

In [ ]:
# extract label from each window
labels = []
for i in cleaned_data:
    label = i[0][19]
    labels.append(label)
labels = np.array(labels)
#labels

In [ ]:
# extract features from each window
features = []
for i in cleaned_data:
    new = np.delete(i, 19, 1)
    features.append(new)
features = np.array(features)
#features

In [ ]:
features[0].shape

In [ ]:
# combine the features and labels
k = list(zip(features, labels))
activity_data = pd.DataFrame(k)
activity_data.columns = ['features', 'labels']
activity_data

In [ ]:
# check the size of activity. The final output of neural net 
# has to have max_index + 1 output
max_index = activity_data['labels'].max()
label_size = int(max_index + 1)

---
## Load data

In [ ]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()
if train_on_gpu:
    print("CUDA is available! Training on GPU.")
else:
    print("CUDA is not available. Training on CPU...")
torch.cuda.current_device()

In [ ]:
# split the data to test, validation, and train
valid_size = 0.1
test_size = 0.1
activity_data.columns = ["features", "labels"]
activity_data_train = activity_data[:int(len(activity_data)*(1-valid_size-test_size))]
activity_data_valid = activity_data[int(len(activity_data)*(1-valid_size-test_size)):int(len(activity_data)*(1-test_size))]
activity_data_test = activity_data[int(len(activity_data)*(1-test_size)):]
# activity_data_train.to_csv("./activity_data_train.csv", encoding='utf-8-sig')
# activity_data_valid.to_csv("./activity_data_valid.csv", encoding='utf-8-sig')
# activity_data_train.to_csv("./activity_data_test.csv", encoding='utf-8-sig')

In [ ]:
# define our dataset in pytorch
class DatasetSpineTrack(Dataset):
    
    def __init__(self, file, transform=None):
        #self.data = pd.read_csv(file_path)
        self.data = file
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        # load image as ndarray type (Height * Width * Channels)
        # be carefull for converting dtype to np.uint8 [Unsigned integer (0 to 255)]
        # in this example, i don't use ToTensor() method of torchvision.transforms
        # so you can convert numpy ndarray shape to tensor in PyTorch (H, W, C) --> (C, H, W)
        
        features = torch.tensor(self.data["features"].iloc[index])
        features = features.view(channel, window_size, 19) 
        labels = torch.tensor(self.data["labels"].iloc[index], dtype=torch.long)
        #print(labels.type())
        
#         if self.transform is not None:
#             image = self.transform(image)
            
        return features, labels

In [ ]:
# construct training and testing dataset in csv
# train_dataset = DatasetSpineTrack("./activity_data_train.csv")
# valid_dataset = DatasetSpineTrack("./activity_data_valid.csv")
# test_dataset = DatasetSpineTrack("./activity_data_test.csv")
train_dataset = DatasetSpineTrack(activity_data_train)
valid_dataset = DatasetSpineTrack(activity_data_valid)
test_dataset = DatasetSpineTrack(activity_data_test)
feature, label = train_dataset.__getitem__(0)
#feature
#label

In [ ]:
# load data
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

---
## Network Architechture

In [ ]:
train_on_gpu = False

In [ ]:
model = models.resnet18(pretrained=False)
# window_size channels
# model.conv1 = torch.nn.Conv2d(window_size, batch_size, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.conv1 = torch.nn.Conv2d(channel, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = torch.nn.Linear(512, label_size, bias=True)
model.add_module("dropout", torch.nn.Dropout(p=0.5))
model = model.double()

# move tensors to GPU is CUDA is available
if train_on_gpu:
    model.cuda()
print(model)

---
## Training the Network

Remember to look at how the training and validation loss decreases over time; if the validation loss ever increases it indicates possible overfitting.

In [ ]:
import time
start_time = time.time()

criterion = nn.CrossEntropyLoss()
#criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# number of epochs to train the model
n_epochs = 30

valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for features, labels in trainloader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            features, labels = features.cuda(), labels.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(features)
        # calculate the batch loss
        loss = criterion(output, labels)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*features.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for features, labels in validloader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            features, labels = features.cuda(), labels.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(features)
        # calculate the batch loss
        loss = criterion(output, labels)
        # update average validation loss 
        valid_loss += loss.item()*features.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(trainloader.sampler)
    valid_loss = valid_loss/len(validloader.sampler)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_Spinetrack_data.pt')
        valid_loss_min = valid_loss
    
    if epoch == 1:
        epoch_running_time = time.time() - start_time
        epoch_sec = epoch_running_time % 60
        epoch_miniute = epoch_running_time / 60
        print("Training time for first epoch: {} min {} sec".format(int(epoch_miniute), int(epoch_sec)))
        
# output running time
running_time = time.time() - start_time
sec = running_time % 60
miniute = running_time / 60
print("Training time: {} min {} sec".format(int(miniute), int(sec)))

###  Load the Model with the Lowest Validation Loss

In [ ]:
model.load_state_dict(torch.load('model_Spinetrack_data.pt'))

---
## Test the Trained Network

Test your trained model on previously unseen data! A "good" result will be a result that gets more than 70% accuracy on these test data.

In [ ]:
# Release all the GPU memory cache that can be freed
torch.cuda.empty_cache() 

### Overall accuracy and each class accuracy

In [ ]:
# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(len(index_dict)))
class_total = list(0. for i in range(len(index_dict)))

criterion = nn.CrossEntropyLoss()
#criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

model.eval()
torch.no_grad()
# iterate over test data
for features, labels in testloader:
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        features, labels = features.cuda(), labels.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(features)
    # calculate the batch loss
    loss = criterion(output, labels)
    # update test loss 
    test_loss += loss.item()*features.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(batch_size):
        try:
            label = labels.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1
        except:
            pass

# average test loss
test_loss = test_loss/len(testloader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(len(index_dict)):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            list(index_dict.keys())[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (list(index_dict.keys())[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

### Overall accuracy (different calculation method)

In [ ]:
# Turn off gradients for validation, saves memory and computations
torch.no_grad()
accuracy = 0
for features, labels in testloader:
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        features, labels = features.cuda(), labels.cuda()
    loss = model(features)
    test_loss += criterion(loss, labels)

#     ps = torch.exp(loss)
    top_p, top_class = loss.topk(1, dim=1)
    equals = top_class == labels.view(*top_class.shape)
    accuracy += torch.mean(equals.type(torch.FloatTensor))
print("Test Accuracy: {:.3f}".format(accuracy/len(testloader)))

In [ ]:
# TODO: Figure out pulling_OneH